# Warlock Project : fichier de test en conditions réelles

In [1]:
# importation des modules
from transformers import WhisperForConditionalGeneration
import os
import pyaudio
import wave
from transformers import pipeline

In [5]:
# on commence par importer le modèle
if False:
    test_model = WhisperForConditionalGeneration.from_pretrained("./model_pre_trained", local_files_only=True)
else:
    # et là, on télécharge le modèle RN pré-entrainé
    test_model = WhisperForConditionalGeneration.from_pretrained("bofenghuang/whisper-small-cv11-french", cache_dir='./model_bofenghuang')     # on dl la version small (à changer potentiellement, dans tous les cas, c'est ce qui est communément utilisé)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

c:\Users\maxen\anaconda3\envs\jcs_warlock\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maxen\Desktop\JCS_mission\warlock\github\jcs-warlock-project-speech-to-text\model_bofenghuang\models--bofenghuang--whisper-small-cv11-french. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.97k [00:00<?, ?B/s]

In [9]:
# puis on allume le micro pour faire un test irl


# Set PYALSA_CARD environment variable to 'default'
os.environ['PYALSA_CARD'] = 'default'

# Initialize the ASR pipeline
asr_pipeline = pipeline(task="automatic-speech-recognition", model="./model_pre_trained")

# Function to capture audio from microphone
def capture_audio(seconds=5, filename="recorded_audio.wav"):
    chunk = 1024  # Number of frames per buffer

    # Initialize PyAudio
    p = pyaudio.PyAudio()

    # Open stream
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=16000,
                    input=True,
                    frames_per_buffer=chunk)

    print("* Recording audio...")

    frames = []

    # Record audio in chunks and save to frames
    for i in range(0, int(16000 / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

    # Close and terminate PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

    print("* Finished recording")

    # Save recorded audio to file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(16000)
    wf.writeframes(b''.join(frames))
    wf.close()

    return filename

# Function to transcribe audio using ASR pipeline
def transcribe_audio(audio_path):
    # Transcribe audio from file path using ASR pipeline
    transcription = asr_pipeline(audio_path)['text']
    return transcription

# Example usage:
if __name__ == "__main__":
    # Capture audio from microphone
    audio_file = capture_audio(seconds=5, filename="recorded_audio.wav")

    # Transcribe captured audio
    result = transcribe_audio(audio_file)

    # Output transcription
    print("Transcription:", result)


OSError: Can't load tokenizer for './model_pre_trained'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './model_pre_trained' is the correct path to a directory containing all relevant files for a WhisperTokenizerFast tokenizer.